In [1]:
from sklearn.utils import shuffle
import pandas as pd
import datetime
import geopandas as gpd
import numpy as np
from pathlib import Path
from tqdm.notebook import tqdm
import io
import plotly.express as px

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf

In [2]:
for model_dir in Path('models').glob('*'):
    model_path = model_dir.joinpath('model.keras')
    model = tf.keras.models.load_model(self.model_dir.joinpath('model.keras'))
    break
model.summary()

ValueError: No model config found in the file at models/binary_crossentropy-237212-7-10-09-64-32/latest.weights.h5.

In [ ]:
all_labels = pd.read_csv(Path('data').joinpath('full_dummies.csv'))
grouped_labels = all_labels.T.groupby(
    all_labels.columns.str.split().str[0],
).max().T
all_labels.shape

In [ ]:
from sentinel_utils.SentinelUtils import normalise

In [ ]:
from time import sleep
from tqdm.notebook import tqdm
for i in tqdm(list(range(1000))):
    sleep(0.1)
    for j in tqdm(list(range(1000)), leave=False):
        sleep(1e-3)

In [ ]:
import geemap
import ee
ee.Authenticate()
ee.Initialize(project='sentinel-treeclassification')


In [ ]:
gdf = gpd.read_file(Path('data').joinpath('full_geometries.json'))
# gdf_points = gdf
# gdf_points.geometry = gdf_points.geometry.centroid

In [ ]:
import importlib
import ee_downloader 
importlib.reload(ee_downloader)

start_date = datetime.datetime(2020, 3, 1)
end_date = datetime.datetime(2024, 3, 1)
ee_downloader.EEDownloader().download_era5(
    gdf, start_date, end_date, Path('weather_data'))

In [ ]:
# df = pd.read_csv(Path('weather_data', 'era5_2020.csv'))
# df.interpolate(method='nearest')

SoilGrids250m: https://git.wur.nl/isric/soilgrids/soilgrids.notebooks/-/blob/master/markdown/access_on_gee.md

In [ ]:
import importlib
import ee_downloader 
importlib.reload(ee_downloader)
from ee_downloader import EEDownloader
from concurrent.futures import ThreadPoolExecutor

chunk_size = 100
chunks = [geometries[i: i + chunk_size] for i in range(0, geometries.shape[0], chunk_size)]


soil_dir = Path.home().joinpath('sentinel_data').joinpath('shards').joinpath('soil')
size = len(chunks)
soil_inputs = (range(size), chunks, [chunk_size]*size, [soil_dir]*size)


with ThreadPoolExecutor(max_workers=12) as pool:
    jobs = list(tqdm(pool.map(EEDownloader().download_soil, *soil_inputs), total=size))

In [ ]:
np.load(soil_file).shape

In [ ]:
np.array().shape

In [ ]:
np.stack([new_data, new_data], axis=-1).shape

In [ ]:
start_date = datetime.datetime(2017, 1, 1)
end_date = datetime.datetime(2020, 1, 1)

In [ ]:
from multiprocessing.pool import ThreadPool as Pool
from tqdm.notebook import tqdm
import sys
from IPython.display import display
from time import sleep

In [ ]:
def test_fun(t):
    a, b = t
    for i in (pbar := tqdm(range(10), leave=False)):
        pbar.set_description(f'Chunk {a}')
        sleep(0.1)

In [ ]:
test_inputs = list(zip(range(10), range(10)))

# pool = Pool(processes=2)
# list(tqdm(pool.starmap(test_fun, test_inputs, chunksize=1), total=len(test_inputs)))
# pool.close()
# pool.join()

In [ ]:
from concurrent.futures import ThreadPoolExecutor

In [ ]:
with ThreadPoolExecutor(max_workers=2) as pool:
    jobs = list(tqdm(pool.map(test_fun, test_inputs), total=len(test_inputs)))


In [ ]:
jobs[0]